In [6]:
import numpy as np
import pandas as pd

files = [f"X1_240501\T2p2\ECG_c{i}.txt" for i in [1,2,3]]

def read_numpy_array_from_file(file_path):
    try:
        data = pd.read_csv(file_path, sep="\t", comment='#', names=['peak', 'ECG', 'smoothed'])
        return data
    except Exception as e:
        print(f"Error reading the numpy array from the file: {e}")
        return None

reading_dfs = [read_numpy_array_from_file(files[i-1]) for i in [1,2,3]]
print(reading_dfs[0])


       peak   ECG  smoothed
0         0  5520 -0.004052
1         0  5518 -0.000369
2         0  5516 -0.000369
3         0  5504 -0.002218
4         0  5572  0.012593
...     ...   ...       ...
39895     0  5620 -0.003257
39896     0  5660  0.007236
39897     0  5538 -0.022049
39898     0  5438 -0.018074
39899     0  5446  0.001447

[39900 rows x 3 columns]
